In [1]:
import sys
sys.path.append('..')

In [ ]:
from Sparse_Baselines import OWC, TFIDF, BM25
from helper import *
import json
import pandas as pd

In [3]:
config = load_config()
train_splits, val_splits, test_splits = load_data(config)

In [ ]:
def owc_eval(test_data):
    OWC_baseline = OWC(test_data)
    score_fcn = OWC_baseline.owc_score
    OWC_baseline.clean_data()
    
    return OWC_baseline.get_results(score_fcn)

def tfidf_eval(test_data):
    TFIDF_baseline = TFIDF(test_data)
    score_fcn = TFIDF_baseline.tfidf_score
    cleaned_descriptions = TFIDF_baseline.clean_data()
    TFIDF_baseline.calc_df(cleaned_descriptions)

    return TFIDF_baseline.get_results(score_fcn)

def bm25_eval(test_data):
    BM25_baseline = BM25(test_data)
    score_fcn = BM25_baseline.bm25_score
    cleaned_descriptions = BM25_baseline.clean_data()
    BM25_baseline.fit(cleaned_descriptions)
    
    return BM25_baseline.get_results(score_fcn)

In [ ]:
def evaluate(name, method):
    results = []
    for i in range(len(train_splits)):
        test_data = test_splits[i]
        correct, total, type_correct, type_count = method(test_data)

        for key, val in type_correct.items():
            type_correct[key] = val*100/type_count[key]
        type_correct.update({"All":correct*100/total})
        results.append(type_correct)

    df = pd.DataFrame(results)
    df.to_csv(name+".csv")

In [4]:
evaluate("owc", owc_eval)
evaluate("tfidf", tfidf_eval)
evaluate("bm25", bm25_eval)